<h1 style='text-aling:center;color:Navy'>  Big Data Systems - Lab #6  </h1>

<h2 style='text-aling:center;color:RED'> Your name is Crystin Rodrick</h2> 

# <span style="color:#3665af">Stream Processing </span><span style="font-size:15px">(Estimated time: 1 hour) </span>
<hr>
The objective of this lab is to introduce the use of Spark streaming


<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Example 1 - Word Count </div>


This is a simple example of using pyspark streaming library. 

You have to run through the example, understand what the code is doing. 
Just follow the instructions and run cells to get a sense of how Spark Streaming works.
By the end of the lab session, make sure you are submitting the outputs of this example.

In [1]:
import findspark

In [2]:
findspark.init('/home/crystin/spark')

In [3]:
from pyspark import SparkContext

In [4]:
from pyspark.streaming import StreamingContext

The first parameter of SparkContext indicates that this application will be run on a single node (not a cluster).
The second parameter is just a name given to this application.

In [16]:
sc = SparkContext('local[2]','NetworkWordCount') 

Now, we initialize the streamig context and set the batch interval to 5 seconds. It means that, each batch will contain the data (RDDs) for 5 seconds

In [17]:
ssc = StreamingContext(sc,5)

Now we want the streaming context to read data from a socket. We need to use a port that is not already in use, like 9999. <br/>
if you get the folloiwng terminal error 'port xxxx is already in use' change the port number.

In [18]:
lines = ssc.socketTextStream('localhost',9999)

Now we want to receive every line of text from the given port and extract words from each line. <br>
We use the flatMap functuion since each line may contain multiple words

In [19]:
words = lines.flatMap(lambda line: line.split(' '))

Now, we need to map each word, to a key value pair, where the key is the word itself and the value is always 1.

In [20]:
pairs = words.map(lambda word:(word,1))

The *reduceByKey* treats the first element of each pair as key and the second element as value!
Then, it groups all values belonging to the same key together.
Now, for couting each word, we simply need to add up all values for that word together

In [21]:
word_counts = pairs.reduceByKey(lambda num1,num2:num1+num2)

Finally, we present the count for each word

In [22]:
word_counts.pprint()

Now, we are ready to start the process. Before running the next cell, open a terminal and type: 
nc -lk 9999
Now you can send messages to port 9999 using the netcat (nc) program.
Go ahead and run the next cell; then go back to terminal and send some messages

In [23]:
ssc.start()

-------------------------------------------
Time: 2018-04-12 12:44:40
-------------------------------------------
('this', 1)
('is', 1)
('a', 1)
('mesage', 1)

-------------------------------------------
Time: 2018-04-12 12:44:45
-------------------------------------------

-------------------------------------------
Time: 2018-04-12 12:44:50
-------------------------------------------
('this', 1)
('is', 1)
('new', 1)
('a', 1)
('message', 1)

-------------------------------------------
Time: 2018-04-12 12:44:55
-------------------------------------------

-------------------------------------------
Time: 2018-04-12 12:45:00
-------------------------------------------
('ths', 1)
('is', 1)
('new', 1)
('a', 1)
('message', 1)

-------------------------------------------
Time: 2018-04-12 12:45:05
-------------------------------------------
('', 1)



When you are done, run the next cell to stop the streaming context. <br>
Note that if you want to run the streaming context again, you need to restart the kernel. You may also need to change the port no

In [24]:
ssc.stop()

---

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Example 2 - Tweet analysis </div>


### Before Continuing

Before we start with this part of the lab, we will need to run the python program *tweetread.py* that reads the tweets and streams those to a local port.

On a text editor open the file *tweetread.py*. We need to configure the following parameters. You should obtain the value for each parameter from your tweeter configuration (review the guide).

<pre style="background-color: #ebece4;padding: 10px;border-left: solid 4px orange;">
# Set up your credentials
# add your credentials between ''
consumer_key=''
consumer_secret=''
access_token =''
access_secret=''
</pre>


Also, you might need to change this port in case you receive a *'port xxxx is already in use'* error.
<pre style="background-color: #ebece4;padding: 10px;border-left: solid 4px orange;">
port = 5555
</pre>


**Open a new shell terminal** and execute the following commands.<br>
*We assume you have your downloaded files for this assignment at **~/lab6**, if you are using another path, replace it below.*
<pre style="background-color: #ebece4;padding: 10px;border-left: solid 4px orange;">
cd ~/lab6 <br>
python3 tweetread.py
</pre>

Once you have this running, return to the notebook and keep going. 


In [66]:
## This command clears the enviroment. If you have any issues, try first to restart the kernel.
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [67]:
import findspark

Update the spark path in the folloiwng cell

In [68]:
findspark.init('/home/crystin/spark')

In [69]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [70]:
totalTweetCount = 0 #stores the number of tweets
totalCharacters = 0 #stores the total length of all tweets

In [71]:
# Can only run this once. restart your kernel for any errors.
#to do so click on Kernel, Resrart & Clear Output
sc = SparkContext()

In [72]:
ssc = StreamingContext(sc, 5 )

- **Note**: If you change the port number on tweetread.py, you need to update it here too.<br>
- Both port numbers (here and tweetread.py) should match. <br>
- You might need to restart the Kernel before continuing.<br>
**Make sure your *tweetread.py* app is running before continue.**

In [73]:
tweets = ssc.socketTextStream("127.0.0.1", 5558)

In [74]:
lengths = tweets.map(lambda tweet: len(tweet)) #get the length of tweets

In [75]:
def updateCounts(rdd):
    try:
        count = rdd.count()
        if(count>0):
            global totalTweetCount
            global totalCharacters
            
            totalTweetCount += count
            totalCharacters += rdd.reduce(lambda len1,len2: len1+len2)
            
        if (totalTweetCount>0):
            print('Total Tweets: ' + str(totalTweetCount) + '\n' +
                  'Total characters: ' + str(totalCharacters) + '\n'
                  'Average Length: ' + str(totalCharacters/totalTweetCount)+
                 '\n------------------------------\n')
        else:
            print('nothing yet')
        
    except Exception as ex:
        print(ex)

In [76]:
lengths.foreachRDD(lambda rdd: updateCounts(rdd))

**Now start the spark streaming context by running the next cell.**
- You should see an updated output every 5 seconds

*Once you are done, execute the stop command in the cell after.*

In [77]:
ssc.start()    

nothing yet
Total Tweets: 100
Total characters: 18349
Average Length: 183.49
------------------------------

Total Tweets: 176
Total characters: 37171
Average Length: 211.19886363636363
------------------------------



In [78]:
ssc.stop()

Total Tweets: 252
Total characters: 57096
Average Length: 226.57142857142858
------------------------------



<div style="font-size:20px;background-color:#A74A54;color:#F1E6E7;padding:10px;">
    Questions
</div>

This code works fine if it is executed on a local machine. 

1. **Explain what will go wrong if it is executed on a cluster.**
2. **How this issues can be solved?** *(No need to modify the code, just explain).*

<br>
<div style="font-size:20px;background-color:#BE6D00;color:#F6EFE5;padding:10px;text-align:center;">
Submit this notebook with the output of the commands, and the answers for the previous two questions.
</div>
<hr style="border: 3px double navy;" >
<br>